In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

import keras
import keras.backend as K
import tensorflow as tf
from keras import applications
from keras.models import Model
from keras.layers import Flatten, Dense, Input,concatenate
from keras.optimizers import Adam
from keras.models import load_model, model_from_json

import random
#from PIL import Image, ImageChops

In [12]:
gen="../input/handwritten-signatures/sample_signature/sample_Signature/genuine"
forg="../input/handwritten-signatures/sample_signature/sample_Signature/forged"

gentr="../input/sigcomp-2009-train/sigcomp 2009 train/Sigcomp 2009 train/genuine"
forgtr="../input/sigcomp-2009-train/sigcomp 2009 train/Sigcomp 2009 train/forgeries"

gent="../input/sigcomp-2009/sigcomp 2009/genuines"
forgt="../input/sigcomp-2009/sigcomp 2009/forgeries"

In [13]:
img_width, img_height, channels = 224, 224, 3

dim = (img_width, img_height)

def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

def returnimages(path,img):
    image=cv2.imread(path+"/"+ img)                  #bringing the image
    image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=to_rgb(image).reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image     

def getfiles(num,gen,forg):
    a=os.listdir(gen)
    b=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    n,m=[],[]
    for i in b:
        if i.endswith(c+".png"):
            n=n+[i]
        elif i.endswith(c+".PNG"):
            n=n+[i]
    for i in a:
        if i.endswith(c+".png"):
            m=m+[i]
        elif i.endswith(c+".PNG"):
            m=m+[i]
    return m.pop(),n,m

def getfiles2(num):
    a=os.listdir(gentr)
    b=os.listdir(forgtr)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    n,m=[],[]
    for i in b:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            n=n+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            n=n+[i]
    for i in a:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            m=m+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            m=m+[i]
    return m.pop(),n,m



In [14]:
def triplet_loss(y_true, y_pred):
    alpha = 0.5
    anchor, positive, negative =y_pred[0,0:512], y_pred[0,512:1024], y_pred[0,1024:1536]
    
    positive_distance = K.mean(K.square(anchor - positive),axis=-1)
    negative_distance = K.mean(K.square(anchor - negative),axis=-1)
    return K.mean(K.maximum(0.0, positive_distance - negative_distance + alpha))

In [25]:
model = applications.vgg19.VGG19(weights='imagenet', include_top=False, pooling='max')

In [26]:
for layer in model.layers[:13]:
    layer.trainable = False

In [27]:
anchor_in = Input(shape=(img_width, img_height, channels))
pos_in = Input(shape=(img_width, img_height, channels))
neg_in = Input(shape=(img_width, img_height, channels))

anchor_out = model(anchor_in)
pos_out = model(pos_in)
neg_out = model(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out],axis=1)

In [28]:
model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

In [29]:
model.compile(optimizer=Adam(lr=0.000005),loss=triplet_loss)

In [30]:
def generator():
    for i in range(1,31):
        if(i<10):
            anc,neg,pos=getfiles(float("0.00"+str(i)),gen,forg)
        else:
            anc,neg,pos=getfiles(float("0.0"+str(i)),gen,forg)
        for i in range(len(neg)):
            for j in range(len(pos)):
                anchor=returnimages(gen,anc)
                positive=returnimages(gen,pos[j])
                negative=returnimages(forg,neg[i])
               # yield ([anc,pos[j],neg[i]],[0])
                yield ([anchor,positive,negative],[0])

In [31]:
for x in range(1):
    model.fit_generator(generator(),steps_per_epoch=200,epochs=3)

Epoch 1/3
200/200 [==============================] - 39s 197ms/step - loss: 0.0844
Epoch 2/3
200/200 [==============================] - 37s 185ms/step - loss: 0.0555
Epoch 3/3
200/200 [==============================] - 45s 226ms/step - loss: 0.0282


In [32]:
model.compile(optimizer=Adam(lr=0.000002),loss=triplet_loss)

In [33]:
def generator2():
    x=["0.001","0.004", "0.005", "0.006", "0.007",
       "0.008", "0.009", "0.010", "0.011", "0.001", "0.011"]
  #  x=["0.001", "0.004", "0.006", "0.010"]

    for k in x:
        anc,neg,pos=getfiles2(k)
        frac=0.95    
        inds = set(random.sample(list(range(len(neg))), int(frac*len(neg))))
        neg = [n for i,n in enumerate(neg) if i not in inds]
    
        for i in range(len(neg)):
            for j in range(len(pos)):
                anchor=returnimages(gentr,anc)
                positive=returnimages(gentr,pos[j])
                negative=returnimages(forgtr,neg[i])
               # yield ([anc,pos[j],neg[i]])
                yield ([anchor,positive,negative],[0])

In [34]:
for x in range(1):
    model.fit_generator(generator2(),steps_per_epoch=32,epochs=11)

Epoch 1/11
32/32 [==============================] - 8s 261ms/step - loss: 0.0471
Epoch 2/11
32/32 [==============================] - 6s 186ms/step - loss: 0.0135
Epoch 3/11
32/32 [==============================] - 6s 185ms/step - loss: 0.0064
Epoch 4/11
32/32 [==============================] - 6s 186ms/step - loss: 0.0078
Epoch 5/11
32/32 [==============================] - 6s 186ms/step - loss: 0.0086
Epoch 6/11
32/32 [==============================] - 6s 185ms/step - loss: 0.0502
Epoch 7/11
32/32 [==============================] - 6s 185ms/step - loss: 0.0000e+00
Epoch 8/11
32/32 [==============================] - 6s 185ms/step - loss: 0.0050
Epoch 9/11
32/32 [==============================] - 6s 186ms/step - loss: 0.0442
Epoch 10/11
32/32 [==============================] - 6s 185ms/step - loss: 2.5446e-04
Epoch 11/11
32/32 [==============================] - 6s 184ms/step - loss: 0.0000e+00


In [35]:
tneg,tpos=0,0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]
for k in x: #the id of signatures you want to check
    #print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    tneg=tneg+len(neg)
    tpos=tpos+len(pos)
print(tneg,tpos)

624 203


In [39]:
os.listdir('../working')

['__notebook_source__.ipynb', 'weights.h5', '.ipynb_checkpoints']

In [40]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
#with open('model_architecture.json', 'w') as f:
#    f.write(model.to_json())

In [41]:
os.listdir('../working')

['__notebook_source__.ipynb',
 'weights.h5',
 'model_weights.h5',
 '.ipynb_checkpoints']

In [42]:
model.load_weights('../working/model_weights.h5')

In [45]:
forg_passed=0
gen_flagged=0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    
    anchor=returnimages(gent,anc)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor,positive,anchor])
        a, p, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist=np.linalg.norm(a-p)
        #print("positive distance is ",dist)
        if(dist>40):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor,negative,anchor])
        a, n, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist=np.linalg.norm(a-n)
        #print("negative distance is ",dist)
        if(dist>40):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)

When k is  0.002
gen flagged -  41.34227 file name is -  NFI-00209002.png
forg passed -  35.487675 file name is -  NFI-08906002.png
forg passed -  36.891907 file name is -  NFI-00305002.png
forg passed -  36.599808 file name is -  NFI-08902002.png
forg passed -  33.22395 file name is -  NFI-08901002.png
When k is  0.008
gen flagged -  40.020687 file name is -  NFI-00806008.png
gen flagged -  44.18793 file name is -  NFI-00803008.png
gen flagged -  42.581528 file name is -  NFI-00808008.png
gen flagged -  41.694736 file name is -  NFI-00807008.png
When k is  0.016
gen flagged -  58.731308 file name is -  NFI-01612016.png
gen flagged -  52.084938 file name is -  NFI-01608016.png
gen flagged -  46.97917 file name is -  NFI-01610016.png
gen flagged -  59.554245 file name is -  NFI-01609016.png
gen flagged -  48.416237 file name is -  NFI-01611016.png
When k is  0.018
gen flagged -  50.565235 file name is -  NFI-01810018.png
gen flagged -  56.809475 file name is -  NFI-01804018.png
gen flag

In [46]:
def getfilest(num,gen,forg):
    a=os.listdir(gen)
    b=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    n,m=[],[]
    for i in b:
        if i.endswith(c+".png"):
            n=n+[i]
        elif i.endswith(c+".PNG"):
            n=n+[i]
    for i in a:
        if i.endswith(c+".png"):
            m=m+[i]
        elif i.endswith(c+".PNG"):
            m=m+[i]
    return m.pop(),m.pop(),n,m

In [47]:
forg_passed=0
gen_flagged=0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc1,anc2,neg,pos=getfilest(k,gent,forgt)
    
    anchor1=returnimages(gent,anc1)
    anchor2=returnimages(gent,anc2)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor1,positive,anchor2])
        a1, p, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist1=np.linalg.norm(a1-p)
        dist2=np.linalg.norm(a2-p)
        dist=(dist1+dist2)/2
        #print("positive distance is ",dist)
        if(dist>40):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor1,negative,anchor2])
        a1, n, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist1=np.linalg.norm(a1-n)
        dist2=np.linalg.norm(a2-n)
        #print("negative distance is ",dist)
        dist=(dist1+dist2)/2
        if(dist>40):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)

When k is  0.002
gen flagged -  42.796905517578125 file name is -  NFI-00209002.png
gen flagged -  41.309852600097656 file name is -  NFI-00211002.png
gen flagged -  40.86908721923828 file name is -  NFI-00202002.png
forg passed -  38.09596252441406 file name is -  NFI-08906002.png
forg passed -  38.87372589111328 file name is -  NFI-00305002.png
forg passed -  38.656707763671875 file name is -  NFI-08902002.png
forg passed -  35.900177001953125 file name is -  NFI-08901002.png
When k is  0.008
gen flagged -  44.00211715698242 file name is -  NFI-00803008.png
When k is  0.016
gen flagged -  52.82816696166992 file name is -  NFI-01612016.png
gen flagged -  40.38568115234375 file name is -  NFI-01601016.png
gen flagged -  48.24708557128906 file name is -  NFI-01608016.png
gen flagged -  43.41183853149414 file name is -  NFI-01610016.png
gen flagged -  55.08385467529297 file name is -  NFI-01609016.png
gen flagged -  44.06134796142578 file name is -  NFI-01611016.png
When k is  0.018
gen 